Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices, randint, choice, sample, random
from copy import copy, deepcopy

import lab9_lib
import numpy as np
import math


In [2]:
fitness = lab9_lib.make_problem(1)
for n in range(10):     #Determina il numero di Fitness call
    ind = choices([0, 1], k=1000)     #genoma casuale di 50 elementi
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

1111010000000111010101101000011001100000010100000111110110011101110111001101101001111100100000000011010001001001111011100010111010100100010101111111000100100010101000000111110001100010001000111001101111111000101000111110001101101010000100111010010110011101000111000010010010101000111110100001101111101011101000101101000011111011011001011111011100011110100000011001001011100000011110110101000100100010000000101001001001110010101000000011101001101100000100110011001111100110111100000010100000011011011100001100111011001110101000000011101110010101011000011110101000101111010001111010000011111000011101101101001101110001001000001110011100000110110001100110011010111100111001100000111110101000011101001111001000011010101110010111100111010110100100000001101110111101000111100101101101001000100000110110001101101101111000100101101110011101100100010100001111001110111011100100111111000000001011100011001011100110110111110111111011000111110101011100010110000111001101110100100111001100011101001100100011011110

# Local Search Algorithm

In [3]:
# GLOBAL PARAMETER #
NUM_LOCI = 1000

POPULATION_SIZE = 10
NEW_OFFSPRING = 30
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .15
NUM_ISLANDS = 3

# . . . . . . #

### Mutation Techniques

In [4]:
def mutation1(ind):     # 1 loci(gene) mutated
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    offspring[pos] = 1 - offspring[pos]     # Se ho T/F-> offspring[pos] = not offspring[pos]
    #assert len(offspring) == NUM_LOCI
    return offspring

def mutation2(ind):     # Reset randomly a random number of loci
    offspring = copy(ind)
    poss = (randint(0, NUM_LOCI-1), randint(0, NUM_LOCI-1))
    offspring = ind[:min(poss)] + [choice([0, 1])for _ in range(max(poss)-min(poss))] + ind[max(poss):]
    #assert len(offspring) == NUM_LOCI

    return offspring

### Recombination Techniques

In [42]:
def one_cut_crossover(ind1, ind2):
    cut_point = randint(0, NUM_LOCI-1)
    offspring = ind1[:cut_point] + ind2[cut_point:]
    #    assert len(offspring) == NUM_LOCI
    return offspring

'''def n_cut_crossover(ind1: fitness, ind2: fitness) -> fitness:
    num_cut = math.ceil(np.random.normal(loc=10, scale=5))
    cut_point = randint(0, NUM_LOCI-1)
    offspring = ind1[:(NUM_LOCI/num_cut)]

    offspring = ind1[:cut_point] + ind2[cut_point:]
    assert len(offspring.genotype) == NUM_LOCI
    return offspring'''

def uniform_crossover(ind1, ind2):
    offspring = [ind1[i] if i % 2 else ind2[i] for i in range(NUM_LOCI)]
    # assert len(offspring) == NUM_LOCI
    return offspring

def uniform_crossover_double(ind1, ind2):
    o1, o2 = ([ind1[i] if i % 2 else ind2[i] for i in range(NUM_LOCI)], [ind2[i] if i % 2 else ind1[i] for i in range(NUM_LOCI)])
    # assert len(offspring) == NUM_LOCI
    return o1, o2

### Parent Selection

In [62]:
def roulette_wheel(population):     # Roulette wheel with same probability
    return choice(population)

def variable_tournament(population):
    
    n_partecipants = randint(2, int(len(population)/2))
    pool = choices(population, k=n_partecipants)
    parent = max(pool, key=lambda i: fitness(i))

    return parent

def static_tournament(pop):

    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champ = max(pool, key=lambda i: fitness(i))
    
    return champ

def best_parent_ever(population):
    champ = max(population, key=lambda i: fitness(i))
    return champ

def genotype_screening(population):     # Select the 2 element with most differnt genotype 
    distance = []
    for i in range(len(population)):
        distance.append([sum(np.bitwise_xor(np.array(population[i]), np.array(population[j]))) for j in range(len(population))])
    
    distance = np.array(distance)
    max_position = np.unravel_index(np.argmax(distance, axis=None), distance.shape)
    
    return population[max_position[0]], population[max_position[1]]

### Survival Selection

In [7]:
def survival_selection(population):
    population.sort(key=lambda i:fitness(i), reverse=True)       # ORDERING FROM BEST TO WORSE
    return population[:POPULATION_SIZE]       # SURVIVAL SELECTION
    
def remove_twin(population):        # Remove TWIN from the population because I belive that they will have the same fitness
    twins = set()
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            if population[i]==population[j]:
                twins.add(j)
    
    new_p = [ind for i, ind in enumerate(population) if i not in twins]
    return new_p



### Island Definition

In [8]:
def island_0(population):   #Standard screening type of island
    return #...

def island_1(population):   #Genotype screening type of island
    offspring = list()
    for counter in range(NEW_OFFSPRING):
        if random() < MUTATION_PROBABILITY:
            # MUTATION 
            p = roulette_wheel(population)
            o = mutation1(p)
        else:
            #CROSS-OVER
            p1 = variable_tournament(population)
            p2 = variable_tournament(population)
            o =  one_cut_crossover(p1, p2)
        offspring.append(o)

    population.extend(offspring)
    population = remove_twin(population)
    population = survival_selection(population)
    return population

def island_2(population):   # Fitness screening type of island
    return #...

def island_3(population):   # Where all the result come togheter
    return #...

## Problem start from HERE ->

In [63]:
fitness = lab9_lib.make_problem(1)

starting_population = [choices([0, 1], k=NUM_LOCI) for _ in range(POPULATION_SIZE)]

In [10]:
print(len(starting_population))

print(starting_population[1]==starting_population[9])

10
False


In [64]:
population = starting_population
island_0 = starting_population
champ_island=[]
galapagos = (island_0, champ_island)
mutation_strategy = (mutation1, mutation2)
recombination_strategy = (one_cut_crossover, uniform_crossover)
parent_selection_strategy = (roulette_wheel, static_tournament, variable_tournament)
survival_selection_strategy = (remove_twin, survival_selection)

while all(fitness(i) != 1 for i in population):     # condizione da sistemare
    # Island 0 #
    offspring = list()
    
    p1, p2 = genotype_screening(population)
    o1, o2 =  uniform_crossover_double(p1, p2)    
    population.extend([o1, o2])

    for counter in range(NEW_OFFSPRING):
        if random() < MUTATION_PROBABILITY:
            # MUTATION 
            p = roulette_wheel(population)
            o = mutation1(p)
        else:
            #CROSS-OVER
            p1 = variable_tournament(population)
            p2 = variable_tournament(population)
            o =  uniform_crossover(p1, p2)
        offspring.append(o)

    population.extend(offspring)
    population = remove_twin(population)
    population = survival_selection(population)
    
    # Island 1 #
    
    # Island 2 #


    print(fitness(population[0]))
print(fitness.calls)

0.538
0.539
0.547
0.547
0.547
0.547
0.548
0.549
0.55
0.55
0.55
0.55
0.551
0.552
0.553
0.554
0.554
0.555
0.556
0.556
0.557
0.557
0.558
0.558
0.559
0.56
0.56
0.561
0.561
0.562
0.563
0.563
0.565
0.565
0.565
0.567
0.568
0.569
0.569
0.57
0.571
0.573
0.573
0.574
0.574
0.575
0.575
0.575
0.575
0.576
0.577
0.577
0.577
0.578
0.578
0.579
0.579
0.58
0.581
0.582
0.582
0.583
0.583
0.583
0.585
0.586
0.586
0.588
0.589
0.589
0.59
0.59
0.592
0.593
0.593
0.593
0.593
0.593
0.594
0.595
0.596
0.597
0.598
0.599
0.6
0.601
0.602
0.603
0.603
0.604
0.604
0.604
0.605
0.606
0.607
0.608
0.608
0.609
0.61
0.61
0.611
0.611
0.613
0.613
0.613
0.613
0.614
0.615
0.615
0.616
0.616
0.617
0.618
0.618
0.618
0.619
0.619
0.62
0.62
0.622
0.623
0.623
0.623
0.623
0.624
0.624
0.625
0.625
0.626
0.627
0.627
0.627
0.628
0.628
0.629
0.629
0.629
0.63
0.63
0.631
0.632
0.632
0.633
0.634
0.634
0.635
0.635
0.636
0.637
0.637
0.637
0.637
0.638
0.638
0.638
0.639
0.64
0.641
0.641
0.641
0.641
0.642
0.642
0.643
0.644
0.644
0.644
0.645
0.646
0.646